# Translation test

We try to translate bible sentences to disembiguate pronoun gender

In [ ]:
%%bash
pip install sentencepiece
pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_fr_en = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")

model_fr_en = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-fr-en")

In [ ]:
def translate_sentence(input_string, model, tokenizer):
  input_ids = tokenizer(input_string, return_tensors="pt").input_ids
  outputs = model.generate(input_ids)
  return tokenizer.decode(outputs[0], skip_special_tokens=True)
    

test_dataset = [
  "Et mon cul, c'est du poulet ?",
  " Je vais tomber dans les pommes."
]

for sentence in test_dataset:
  print(sentence + ' -> ' + translate_sentence(sentence, model_fr_en, tokenizer_fr_en))

Et mon cul, c'est du poulet ? -> And my ass is chicken?
 Je vais tomber dans les pommes. -> I'm gonna fall in the apples.


# Lib and dataset import

In [ ]:
#load libs
from google.colab import files
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
np.warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import re
import pprint   # For proper print of sequences.
# import treetaggerwrapper
# from googletrans import Translator
from nltk.tokenize import sent_tokenize 
import glob

In [ ]:
sentence_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/sentence_dataset/sentence_dataset.csv', index_col=0)
sentence_dataset

,chapter_number,page_number,line_number,sentence_number,sentence
0,1.0,1.0,1,0,"Au commencement, Dieu créa les cieux et la terre."
1,1.0,1.0,2,0,La terre était informe et vide: il y avait des...
2,1.0,1.0,3,0,Dieu dit: Que la lumière soit!
3,1.0,1.0,3,1,Et la lumière fut.
4,1.0,1.0,4,0,Dieu vit que la lumière était bonne; et Dieu s...
...,...,...,...,...,...
40855,66.0,22.0,19,0,et si quelqu'un retranche quelque chose des pa...
40856,66.0,22.0,20,0,"Celui qui atteste ces choses dit: Oui, je vien..."
40857,66.0,22.0,20,1,Amen!
40858,66.0,22.0,20,2,"Viens, Seigneur Jésus!"


# Test translation on a small sample

## Sample

In [ ]:
sample_100 = sentence_dataset[:10]

## FR TO EN

In [ ]:
sample_100['en_translation'] = sample_100.apply(lambda row: translate_sentence(row['sentence'], model_fr_en, tokenizer_fr_en), axis=1)
sample_100

,chapter_number,page_number,line_number,sentence_number,sentence,en_translation
0,1.0,1.0,1,0,"Au commencement, Dieu créa les cieux et la terre.",In the beginning God created the heavens and t...
1,1.0,1.0,2,0,La terre était informe et vide: il y avait des...,The earth was informed and empty: there was da...
2,1.0,1.0,3,0,Dieu dit: Que la lumière soit!,"God said, Let the light be!"
3,1.0,1.0,3,1,Et la lumière fut.,And the light was.
4,1.0,1.0,4,0,Dieu vit que la lumière était bonne; et Dieu s...,God saw that the light was good; and God separ...
5,1.0,1.0,5,0,"Dieu appela la lumière jour, et il appela les ...","God called the light day, and he called darkne..."
6,1.0,1.0,5,1,"Ainsi, il y eut un soir, et il y eut un matin:...","So there was evening, and there was morning: i..."
7,1.0,1.0,6,0,Dieu dit: Qu'il y ait une étendue entre les ea...,"God said, ""Let there be an expanse between the..."
8,1.0,1.0,7,0,"Et Dieu fit l'étendue, et il sépara les eaux q...","And God made the expanse, and separated the wa..."
9,1.0,1.0,7,1,Et cela fut ainsi.,And so it was.


## EN TO FR

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_en_fr = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

model_en_fr = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

In [ ]:
translate_sentence('this is a simple test', model_en_fr, tokenizer_en_fr)

"c'est un test simple"

In [ ]:
sample_100['en_fr_translation'] = sample_100.apply(lambda row: translate_sentence(row['en_translation'], model_en_fr, tokenizer_en_fr), axis=1)
sample_100

,chapter_number,page_number,line_number,sentence_number,sentence,en_translation,en_fr_translation
0,1.0,1.0,1,0,"Au commencement, Dieu créa les cieux et la terre.",In the beginning God created the heavens and t...,"Au commencement, Dieu créa les cieux et la terre."
1,1.0,1.0,2,0,La terre était informe et vide: il y avait des...,The earth was informed and empty: there was da...,La terre était informée et vide: il y avait de...
2,1.0,1.0,3,0,Dieu dit: Que la lumière soit!,"God said, Let the light be!",Dieu dit: Que la lumière soit!
3,1.0,1.0,3,1,Et la lumière fut.,And the light was.,Et la lumière l'était.
4,1.0,1.0,4,0,Dieu vit que la lumière était bonne; et Dieu s...,God saw that the light was good; and God separ...,"Dieu vit que la lumière était bonne, et Dieu s..."
5,1.0,1.0,5,0,"Dieu appela la lumière jour, et il appela les ...","God called the light day, and he called darkne...","Dieu appela la lumière jour, et il appela la n..."
6,1.0,1.0,5,1,"Ainsi, il y eut un soir, et il y eut un matin:...","So there was evening, and there was morning: i...","Il y eut donc un soir, et il y eut un matin: c..."
7,1.0,1.0,6,0,Dieu dit: Qu'il y ait une étendue entre les ea...,"God said, ""Let there be an expanse between the...",Dieu dit: Qu'il y ait une étendue entre les ea...
8,1.0,1.0,7,0,"Et Dieu fit l'étendue, et il sépara les eaux q...","And God made the expanse, and separated the wa...","Et Dieu fit l'étendue, et sépara les eaux qui ..."
9,1.0,1.0,7,1,Et cela fut ainsi.,And so it was.,Et c'était le cas.


## Gender Swap Test

In [ ]:
translate_sentence('this is a simple test', model_en_fr, tokenizer_en_fr)

"c'est un test simple"

In [ ]:
translate_sentence('ceci est un test simple', model_fr_en, tokenizer_fr_en)

'This is a simple test'

In [ ]:
masc_gender_sentence = "Dieu créa l'homme à son image, il le créa à l'image de Dieu, il créa l'homme et la femme."

In [ ]:
masc_english_sentence = translate_sentence(masc_gender_sentence, model_fr_en, tokenizer_fr_en)
masc_english_sentence

'God created man in his image, he created him in the image of God, he created man and woman.'

In [ ]:
import re

fem_english_sentence = re.sub(r'\bhe', 'she', masc_english_sentence)
fem_english_sentence

'God created man in his image, she created him in the image of God, she created man and woman.'

In [ ]:
fem_gender_sentence = translate_sentence(fem_english_sentence, model_en_fr, tokenizer_en_fr)
fem_gender_sentence

"Dieu créa l'homme à son image, elle le créa à l'image de Dieu, elle créa l'homme et la femme."

## Gender Swap sample

In [ ]:
def gender_swap_sentence(masc_gender_sentence):
  masc_english_sentence = translate_sentence(masc_gender_sentence, model_fr_en, tokenizer_fr_en)
  fem_english_sentence = re.sub(r'\bhe\s', 'she', masc_english_sentence)
  fem_english_sentence = re.sub(r'\bHe\s', 'She', fem_english_sentence)

  fem_gender_sentence = translate_sentence(fem_english_sentence, model_en_fr, tokenizer_en_fr)
  return fem_gender_sentence

gender_swap_sentence("il s'habilla et partit")

"Elle s'est habillée et est partie."

In [ ]:
sample_100_gender_swap = sentence_dataset[:100]

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

sample_100_gender_swap['gender_swapped'] = sample_100_gender_swap.progress_apply(lambda row: gender_swap_sentence(row['sentence']), axis=1)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
sample_100_gender_swap

,chapter_number,page_number,line_number,sentence_number,sentence,gender_swapped
0,1.0,1.0,1,0,"Au commencement, Dieu créa les cieux et la terre.","Au commencement, Dieu créa les cieux et la terre."
1,1.0,1.0,2,0,La terre était informe et vide: il y avait des...,La terre était informée et vide: il y avait de...
2,1.0,1.0,3,0,Dieu dit: Que la lumière soit!,Dieu dit: Que la lumière soit!
3,1.0,1.0,3,1,Et la lumière fut.,Et la lumière l'était.
4,1.0,1.0,4,0,Dieu vit que la lumière était bonne; et Dieu s...,"Dieu vit que la lumière était bonne, et Dieu s..."
...,...,...,...,...,...,...
95,1.0,3.0,17,2,C'est à force de peine que tu en tireras ta no...,C'est dur pour toi d'en sortir ta nourriture t...
96,1.0,3.0,18,0,"il te produira des épines et des ronces, et tu...",Elle produira des épines et des épines pour vo...
97,1.0,3.0,19,0,C'est à la sueur de ton visage que tu mangeras...,C'est avec la sueur de ton visage que tu mange...
98,1.0,3.0,20,0,Adam donna à sa femme le nom d'Eve: car elle a...,"Adam appela sa femme Eve, car elle était la mè..."


In [ ]:
sample_100_gender_swap.to_csv('/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/sentence_dataset/gender_swapped_sentence_100.csv')

## Muliple translations

In [ ]:
def deep_fry_sentence(input_sentence, fry_count):
  # fem_english_sentence = re.sub(r'\bhe\s', 'she', masc_english_sentence)
  # fem_english_sentence = re.sub(r'\bHe\s', 'She', fem_english_sentence)

  for fry_instance in tqdm(range(0, fry_count)):
    input_sentence = translate_sentence(input_sentence, model_fr_en, tokenizer_fr_en)
    input_sentence = translate_sentence(input_sentence, model_en_fr, tokenizer_en_fr)
    
  return input_sentence

deep_fry_sentence("il s'habilla et partit", 10)

  0%|          | 0/10 [00:00<?, ?it/s]

"Il s'est habillé et est parti."

In [ ]:
sample_100 = sentence_dataset[:10]

In [ ]:
sample_100 = sentence_dataset[:10]

sample_100['deep_fried'] = sample_100.progress_apply(lambda row: deep_fry_sentence(row['sentence'], 10), axis=1)
sample_100

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

,chapter_number,page_number,line_number,sentence_number,sentence,deep_fried
0,1.0,1.0,1,0,"Au commencement, Dieu créa les cieux et la terre.","Au commencement, Dieu créa les cieux et la terre."
1,1.0,1.0,2,0,La terre était informe et vide: il y avait des...,La terre était informée et vide: il y avait de...
2,1.0,1.0,3,0,Dieu dit: Que la lumière soit!,Dieu dit: Que la lumière soit!
3,1.0,1.0,3,1,Et la lumière fut.,Et la lumière l'était.
4,1.0,1.0,4,0,Dieu vit que la lumière était bonne; et Dieu s...,"Dieu vit que la lumière était bonne, et Dieu s..."
5,1.0,1.0,5,0,"Dieu appela la lumière jour, et il appela les ...","Dieu appela la lumière jour, et il appela la n..."
6,1.0,1.0,5,1,"Ainsi, il y eut un soir, et il y eut un matin:...","Il y eut donc un soir, et il y eut un matin: c..."
7,1.0,1.0,6,0,Dieu dit: Qu'il y ait une étendue entre les ea...,Dieu dit: Qu'il y ait une étendue entre les ea...
8,1.0,1.0,7,0,"Et Dieu fit l'étendue, et il sépara les eaux q...","Et Dieu fit l'étendue, et sépara les eaux qui ..."
9,1.0,1.0,7,1,Et cela fut ainsi.,Et c'était le cas.


In [ ]:
translate_sentence("God is dressed for the occasion.", model_en_fr, tokenizer_en_fr)

"Dieu est habillé pour l'occasion."

In [ ]:
translate_sentence("Laetitia is dressed for the occasion.", model_en_fr, tokenizer_en_fr)

"Laetitia est habillée pour l'occasion."

In [ ]:
translate_sentence("Louis is dressed for the occasion.", model_en_fr, tokenizer_en_fr)

"Louis est habillé pour l'occasion."

# Noun extraction

## Raw bible sentences

In [134]:
sentence_dataset = sentence_dataset[~(sentence_dataset.sentence == '-')]
sentence_dataset

,chapter_number,page_number,line_number,sentence_number,sentence
0,1.0,1.0,1,0,"Au commencement, Dieu créa les cieux et la terre."
1,1.0,1.0,2,0,La terre était informe et vide: il y avait des...
2,1.0,1.0,3,0,Dieu dit: Que la lumière soit!
3,1.0,1.0,3,1,Et la lumière fut.
4,1.0,1.0,4,0,Dieu vit que la lumière était bonne; et Dieu s...
...,...,...,...,...,...
40855,66.0,22.0,19,0,et si quelqu'un retranche quelque chose des pa...
40856,66.0,22.0,20,0,"Celui qui atteste ces choses dit: Oui, je vien..."
40857,66.0,22.0,20,1,Amen!
40858,66.0,22.0,20,2,"Viens, Seigneur Jésus!"


## Treetagger model and tagged bible dataset import

In [ ]:
%%bash
cd '/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/treetagger'
# Download the tagger package for your system (PC-Linux, Mac OS-X, ARM64, ARMHF, ARM-Android, PPC64le-Linux).
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
tar -xzvf tree-tagger-linux-3.2.4.tar.gz
# Download the tagging scripts into the same directory.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz
gunzip tagger-scripts.tar.gz
# Download the installation script install-tagger.sh.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/install-tagger.sh
# Download the parameter files for the languages you want to process.
# list of all files (parameter files) https://cis.lmu.de/~schmid/tools/TreeTagger/#parfiles
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/english.par.gz

wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/french.par.gz

sh install-tagger.sh
cd ..
sudo pip install treetaggerwrapper

bin/
bin/train-tree-tagger
bin/separate-punctuation
bin/tree-tagger
bin/tree-tagger-flush
cmd/
cmd/lookup.perl
COPYRIGHT
doc/
doc/sigdat95.pdf
doc/nemlap94.pdf
FILES
README
Release-Notes

TreeTagger version for PC-Linux installed.
Tagging scripts installed.
English parameter file installed.
French parameter file installed.
Tagging scripts installed.
Path variables modified in tagging scripts.

You might want to add /content/drive/MyDrive/Colab Notebooks/ressources/la beubli/treetagger/cmd and /content/drive/MyDrive/Colab Notebooks/ressources/la beubli/treetagger/bin to the PATH variable so that you do not need to specify the full path to run the tagging scripts.

  Created wheel for treetaggerwrapper: filename=treetaggerwrapper-2.3-py3-none-any.whl size=40772 sha256=d78235f2ddf4ca2ee0140f0fee5f53fd3b08334213790fe3ec1c86d22abc20f8
  Stored in directory: /root/.cache/pip/wheels/a0/93/50/47079639c52033b2e2b865a59654eea6832068149414cb78a5
Successfully built treetaggerwrapper


--2022-03-20 00:37:48--  https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
Resolving cis.lmu.de (cis.lmu.de)... 129.187.148.72, 2001:4ca0:4f01::5
Connecting to cis.lmu.de (cis.lmu.de)|129.187.148.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1889240 (1.8M) [application/x-gzip]
Saving to: ‘tree-tagger-linux-3.2.4.tar.gz.14’

     0K .......... .......... .......... .......... ..........  2% 1.31M 1s
    50K .......... .......... .......... .......... ..........  5% 2.23M 1s
   100K .......... .......... .......... .......... ..........  8% 29.2M 1s
   150K .......... .......... .......... .......... .......... 10% 2.96M 1s
   200K .......... .......... .......... .......... .......... 13% 20.3M 1s
   250K .......... .......... .......... .......... .......... 16% 53.8M 0s
   300K .......... .......... .......... .......... .......... 18% 35.8M 0s
   350K .......... .......... .......... .......... .......... 21% 2.97M 0s
   40

In [ ]:
!pip install --upgrade treetaggerwrapper
!pip install googletrans==4.0.0-rc1

     |████████████████████████████████| 55 kB 1.8 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 1.3 MB 32.1 MB/s 
     |████████████████████████████████| 53 kB 1.3 MB/s 
     |████████████████████████████████| 65 kB 3.1 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17416 sha256=fc20b4b44496cb01887a16bef659a02082757b7c2b29c88921cd4fa6cd1a41c8
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans


## Load pickled database

In [149]:
import pickle
import treetaggerwrapper

with open('/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/tagged_french_bible/tagged_french_bible.pickle', 'rb') as fp:
  tag_list = pickle.load(file=fp)
  fp.close()

## Extract Tag Nouns from taglist

In [176]:
# import treetaggerwrapper

tag_list_small = tag_list[:100]

for tag in tag_list_small:
  # print(tag)
  for inner_tag in tag:
  #   print(type(inner_tag))

#   # print(type(tag))
    if (isinstance(inner_tag, treetaggerwrapper.Tag)):
      if (inner_tag.pos == "NOM"):
        print(inner_tag.word)

#   # break

commencement
Dieu
cieux
terre
terre
ténèbres
surface
abîme
esprit
Dieu
eaux
Dieu
lumière
lumière
Dieu
lumière
Dieu
lumière
ténèbres
Dieu
lumière
jour
ténèbres
nuit
soir
matin
jour
Dieu
étendue
eaux
eaux
eaux
Dieu
étendue
eaux
étendue
eaux
étendue
Dieu
étendue
ciel
soir
matin
jour
Dieu
eaux
ciel
lieu
Dieu
terre
amas
eaux
mers
Dieu
Dieu
terre
verdure
herbe
semence
arbres
fruit
espèce
semence
terre
terre
verdure
herbe
semence
espèce
arbres
fruit
semence
espèce
Dieu
soir
matin
jour
Dieu
luminaires
étendue
ciel
jour
nuit
signes
époques
jours
années
luminaires
étendue
ciel
terre
Dieu
luminaires
luminaire
jour
luminaire
nuit
étoiles
Dieu
étendue
ciel
terre
jour
nuit
lumière
ténèbres
Dieu
soir
matin
jour
Dieu
eaux
abondance
animaux
oiseaux
terre
étendue
ciel
Dieu
poissons
vivants
eaux
abondance
espèce
oiseau
espèce
Dieu
Dieu
eaux
mers
oiseaux
terre
soir
matin
jour
Dieu
terre
animaux
espèce
bétail
reptiles
animaux
espèce
Dieu
animaux
terre
espèce
bétail
espèce
reptiles
terre
espèce
Dieu
Dieu
ho